In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SparkSession
from pyspark.sql import functions as fun 

# Create a SparkSession
spark = SparkSession.builder.appName("StructuredStreaming").getOrCreate()

# Monitor the logs directory for new log data, and read in the raw lines as accessLines
accessLines = spark.readStream.text("../resources/logs")

# Parse out the common log format to a DataFrame
contentSizeExp = r'\s(\d+)$'
statusExp = r'\s(\d{3})\s'
generalExp = r'\"(\S+)\s(\S+)\s*(\S*)\"'
timeExp = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
hostExp = r'(^\S+\.[\S+\.]+\S+)\s'

logsDF = accessLines.select(fun.regexp_extract('value', hostExp, 1).alias('host'),
                         fun.regexp_extract('value', timeExp, 1).alias('timestamp'),
                         fun.regexp_extract('value', generalExp, 1).alias('method'),
                         fun.regexp_extract('value', generalExp, 2).alias('endpoint'),
                         fun.regexp_extract('value', generalExp, 3).alias('protocol'),
                         fun.regexp_extract('value', statusExp, 1).cast('integer').alias('status'),
                         fun.regexp_extract('value', contentSizeExp, 1).cast('integer').alias('content_size'))

# Keep a running count of every access by status code
statusCountsDF = logsDF.groupBy(logsDF.status).count()

# Kick off our streaming query, dumping results to the console
query = ( statusCountsDF.writeStream.outputMode("complete").format("console").queryName("counts").start() )

# Run forever until terminated
query.awaitTermination()

# Cleanly shut down the session
spark.stop()



your 131072x1 screen size is bogus. expect trouble
24/12/23 14:03:40 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/23 14:03:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 14:03:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/23 14:03:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/23 14:03:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-737ce4f4-44a4-4135-a1a1-366bdc471cdb. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp